In [2]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
import torch
import pyvene as pv
import pprint
from task import MetaLearningTask, FineTuningTask
from transformers import BatchEncoding, PretrainedConfig
import jax.random as jr
from torch2jax import j2t, t2j
import dataclasses
import pyreft
import nvidia_smi
from models.mamba import MambaLMHeadModel
from dataclasses import dataclass
from mamba_ssm.utils.generation import InferenceParams
from task import DynamicIntervenableModel, TuneConfig, LoraConfig, ReftConfig
import gc
from models.gpt import GPT, GPTConfig
from models.mamba import MambaLMHeadModel, MambaConfig
from pympler import asizeof
from data.hmm import CompositionalHMMDataset, CompositionalHMMDatasetConfig
import lightning as L
from torch.nn import init
from peft import PeftModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# gpt : 9qs3bntc
# mamba: e72ebe43

In [91]:
task = FineTuningTask(
    TuneConfig(
        "9qs3bntc", LoraConfig(r=16, target_modules="all-linear"), [[1, 0], [2, 1]]
    )
).cuda()

number of parameters: 4.75M
Loaded dataset : (11288/1000)
Loaded checkpoing : epoch=894,step=40250.ckpt


In [93]:
task = FineTuningTask(
    TuneConfig(
        "9qs3bntc",
        ReftConfig(
            low_rank_dimension=12, layers=[3], t_slice=(0,1,1), component="output"
        ),
        [[1, 0], [2, 1]],
        batch_size=32,
    )
)
task.model.to("cuda")

number of parameters: 4.75M
Loaded dataset : (11288/1000)
Loaded checkpoing : epoch=894,step=40250.ckpt


DynamicIntervenableModel(
  (model): GPT(
    (transformer): ModuleDict(
      (wte): Embedding(51, 256)
      (wpe): Embedding(300, 256)
      (drop): Dropout(p=0.0, inplace=False)
      (h): ModuleList(
        (0-5): 6 x Block(
          (ln_1): LayerNorm()
          (attn): CausalSelfAttention(
            (c_attn): Linear(in_features=256, out_features=768, bias=True)
            (c_proj): Linear(in_features=256, out_features=256, bias=True)
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
          )
          (ln_2): LayerNorm()
          (mlp): MLP(
            (c_fc): Linear(in_features=256, out_features=1024, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=1024, out_features=256, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm()
    )
    (lm_head): Linear(in_features=256, out_features=51, bia

In [59]:
task.model.interventions["comp.transformer.h[3].output.unit.pos.nunit.1#0"][
    0
].learned_source.weight = torch.nn.Parameter(
    task.model.interventions["comp.transformer.h[3].output.unit.pos.nunit.1#0"][
        0
    ].rotate_layer.weight.T
)

init.constant_(
    task.model.interventions["comp.transformer.h[3].output.unit.pos.nunit.1#0"][
        0
    ].learned_source.bias,
    0,
);

In [90]:
R = task.model.interventions['comp.transformer.h[3].output.unit.pos.nunit.1#0'][0].rotate_layer.weight
(R.T @ R)

tensor([[ 1.0000e+00,  0.0000e+00, -3.3528e-08,  4.4703e-08,  1.3039e-08,
          3.7253e-09, -2.9802e-08, -7.4506e-09,  2.7940e-08,  0.0000e+00,
          3.3528e-08,  3.9116e-08],
        [ 0.0000e+00,  1.0000e+00,  7.4506e-09, -9.3132e-10,  1.8626e-08,
          1.1176e-08, -9.3132e-09, -1.4901e-08,  0.0000e+00, -9.3132e-09,
          1.0245e-08,  1.1642e-08],
        [-3.3528e-08,  7.4506e-09,  1.0000e+00, -1.8626e-09,  5.0291e-08,
          1.4901e-08, -3.9116e-08,  1.7695e-08, -1.4901e-08, -7.4506e-09,
         -2.6077e-08, -1.8626e-08],
        [ 4.4703e-08, -9.3132e-10, -1.8626e-09,  1.0000e+00, -1.6764e-08,
          2.2352e-08,  9.3132e-10, -1.1176e-08,  1.0245e-08, -1.3039e-08,
          2.0489e-08, -7.4506e-09],
        [ 1.3039e-08,  1.8626e-08,  5.0291e-08, -1.6764e-08,  1.0000e+00,
          1.8626e-09,  1.4901e-08,  2.9802e-08, -1.3970e-08, -7.4506e-09,
          7.4506e-09,  9.3132e-09],
        [ 3.7253e-09,  1.1176e-08,  1.4901e-08,  2.2352e-08,  1.8626e-09,
      

In [73]:
list(task.model.named_parameters())

[('comp.transformer.h[3].output.unit.pos.nunit.1#0.rotate_layer.parametrizations.weight.original',
  Parameter containing:
  tensor([[-0.9990,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0060, -0.9990,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0060,  0.0060, -0.9990,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0060, -0.0060,  0.0060,  ...,  0.0060, -0.0060, -0.0060],
          [-0.0060, -0.0060,  0.0060,  ...,  0.0060, -0.0060, -0.0060],
          [ 0.0060, -0.0060,  0.0060,  ...,  0.0060, -0.0060, -0.0060]],
         device='cuda:0', requires_grad=True)),
 ('comp.transformer.h[3].output.unit.pos.nunit.1#0.learned_source.weight',
  Parameter containing:
  tensor([[-0.0413, -0.0251,  0.0241,  ..., -0.0135,  0.0376,  0.0438],
          [-0.0238,  0.0552,  0.0302,  ...,  0.0247,  0.0336,  0.0160],
          [ 0.0167, -0.0191, -0.0504,  ...,  0.0280,  0.0102, -0.0258],
          ...,
          [-0.0030, -0.0286,  0.0357,  ...,  0.0465

In [62]:
list(task.model.named_parameters())

[('comp.transformer.h[3].output.unit.pos.nunit.1#0.rotate_layer.parametrizations.weight.original',
  Parameter containing:
  tensor([[-1.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0., -1.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0., -1.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]], device='cuda:0',
         requires_grad=True)),
 ('comp.transformer.h[3].output.unit.pos.nunit.1#0.learned_source.weight',
  Parameter containing:
  tensor([[-0.0354, -0.0311,  0.0181,  ..., -0.0195,  0.0317,  0.0498],
          [-0.0298,  0.0612,  0.0363,  ...,  0.0307,  0.0396,  0.0100],
          [ 0.0227, -0.0251, -0.0564,  ...,  0.0221,  0.0042, -0.0198],
          ...,
          [-0.0090, -0.0229,  0.0418,  ...,  0.0525, -0.0131,  0.0503],
          [ 0.0123, -0.0524,  0.0611,  ..., -0.0180, -0.0577,  0.0412],
          [ 0.0014, -0.0016,  0.0104,  ...,  0.0550

In [53]:
list(task.model.named_parameters())

[('comp.transformer.h[3].output.unit.pos.nunit.1#0.rotate_layer.parametrizations.weight.original',
  Parameter containing:
  tensor([[-0.9990,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0060, -0.9990,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0060,  0.0060, -0.9990,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0060,  0.0060,  0.0060,  ..., -0.0060,  0.0060, -0.0060],
          [ 0.0060,  0.0060,  0.0060,  ..., -0.0060,  0.0060, -0.0060],
          [-0.0060, -0.0060, -0.0060,  ...,  0.0060, -0.0060,  0.0060]],
         device='cuda:0', requires_grad=True)),
 ('comp.transformer.h[3].output.unit.pos.nunit.1#0.learned_source.weight',
  Parameter containing:
  tensor([[-0.0253,  0.0354, -0.0181,  ...,  0.0280,  0.0232,  0.0562],
          [-0.0326,  0.0087,  0.0891,  ..., -0.0089, -0.0617,  0.0347],
          [ 0.0508, -0.0398,  0.0063,  ...,  0.2130,  0.1006,  0.0563],
          ...,
          [-0.0641,  0.0095,  0.1182,  ...,  0.0080

In [86]:
trainer = L.Trainer(
    max_steps=100,
    accelerator="gpu",
    enable_checkpointing=False,
    val_check_interval=1000,
    reload_dataloaders_every_n_epochs=1,
    check_val_every_n_epoch=None,
)

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mila/l/leo.gagnon/latent_control/venv/lib/pyth ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [87]:
trainer.fit(task)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                     | Params | Mode 
------------------------------------------------------------------
0 | model        | DynamicIntervenableModel | 6.2 K  | train
  | other params | n/a                      | 4.8 M  | n/a  
------------------------------------------------------------------
0         Trainable params
4.8 M     Non-trainable params
4.8 M     Total params
19.316    Total estimated model params size (MB)
1         Modules in train mode
86        Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [88]:
inp = torch.randn((1, 256), device="cuda")

out = task.model.interventions["comp.transformer.h[3].output.unit.pos.nunit.1#0"][0](
    inp
)

print(torch.sum((inp - out) ** 2))

tensor(17.1549, device='cuda:0', grad_fn=<SumBackward0>)


In [94]:
task.model.model.cuda();

seqs = task.full_data.__getitems__(torch.arange(32), length=100)[0]

out_reft = task.model(seqs)

out_normal = task.model.model(seqs)

print(((out_reft - out_normal)**2).sum())

tensor(4.7273e-06, device='cuda:0', grad_fn=<SumBackward0>)


In [92]:
task.model.cuda();

seqs = task.full_data.__getitems__(torch.arange(32), length=100)[0]

out_lora = task.model(seqs)

with task.model.disable_adapter():
    out_normal = task.model(seqs)

print(((out_lora - out_normal)**2).sum())

tensor(0., device='cuda:0', grad_fn=<SumBackward0>)


In [19]:
out = task.model(seqs)

In [17]:
out_reft.shape

torch.Size([32, 100, 51])

In [31]:
import nvidia_smi

nvidia_smi.nvmlInit()

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate

info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

nvidia_smi.nvmlShutdown()

Total memory: 34359738368
Free memory: 29704519680
Used memory: 4655218688


In [28]:
del logits
gc.collect()
torch.cuda.empty_cache()